In [1]:
import pandas as pd 
import numpy as np
import itertools
import geopandas as gpd
from my_azure_storage import *
from osrm_api import *
from add_location import add_ps_location
from disabled_passenger_api import dispatch_data_loader

- 출발 운행 지역 : 서울(11)
- 도착 운행 지역 : 서울(11), 서울 인근 경기 지역 12곳, 인천공항
- 예외적 운행지역 : 부천(3105), 김포(3123), 양주(3126), 고양(3110), 의정부(3103), 남양주(3113), 구리(3112), 하남(3118), 과천(3111), 안양(3104), 광명(3106), 성남(3102), 인천공항-운서동(2301064)
- 동에서 동으로 이동하는 경우 제외

In [2]:
# def region_filter(subdata):
#     #같은 동에서 같은 동으로 이동하는 경우 제외
#     subdata = subdata.loc[[i!=j for i,j in zip(subdata["출발 행정동 코드"].tolist(), subdata["도착 행정동 코드"].tolist())]]
    
#     ### 출발 필터링
#     #출발 행정동 코드 앞 두 자리 추출을 위해 '문자형'으로 변경
#     origin_code = list(map(str,subdata["출발 행정동 코드"]))

#     #출발은 서울에서만 가능
#     subdata = subdata.loc[list(map(lambda data: data[:2] == "11", origin_code))]
    
#     ### 도착 필터링
#     dest_code = list(map(str, subdata["도착 행정동/ 코드"]))
    
#     # 도착지 체크 함수
#     def check_destination(self):
#         return any([self[:4] == "3105", self[:4] == "3123", self[:4] == "3126", self[:4] == "3110", self[:4] == "3103", self[:4] == "3113", 
#                 self[:4] == "3112", self[:4] == "3118", self[:4] == "3111", self[:4] == "3104", self[:4] == "3106", self[:4] == "3102",
#                 self == "2301064", self[:2] == "11"])
    
#     subdata = subdata.loc[list(map(lambda data: check_destination(data), dest_code))]
#     return subdata


# main_data = pd.DataFrame()
# for i in range(24):
#     if i < 10:
#         sub_data = pd.read_csv(f"rawdata/생활이동_행정동_2022.02_0{i}시.csv", encoding="cp949")
#     else: 
#         sub_data = pd.read_csv(f"rawdata/생활이동_행정동_2022.02_{i}시.csv", encoding="cp949")
        
#     sub_data = region_filter(sub_data)
#     main_data = pd.concat([main_data, sub_data])
    
# main_data = main_data.replace("*",1)
# main_data = main_data[["출발 행정동 코드", "도착 행정동 코드", "이동인구(합)"]]
# main_data["이동인구(합)"] = list(map(float,main_data["이동인구(합)"]))

# main_data = main_data.groupby(["출발 행정동 코드", "도착 행정동 코드"]).sum().reset_index()

# #일 별 이동인구(합) 2월 1일~28일
# main_data["이동인구(합)"] = main_data["이동인구(합)"]/28

In [2]:
def generate_fake_passenger_O_D(fake_num=5):
    #KT 이동량 데이터 기준 일 평균 서울지역 읍면동 별 유동 인구 RAW DATA
    extradata_ct = connect_container("extradata")
    main_data = load_json_trans_data("fake_passenger_raw_data", extradata_ct)

    #서울시 장애인 인구 4% 적용?? -> 약 10배 0.0035(0.35%), 약 5배 0.0017(0.17%)
    if fake_num == 5:
        num = 0.0017
    elif fake_num == 10:
        num = 0.0035
    main_data["이동인구(합)"] = main_data["이동인구(합)"] * num

    #포아송 분포로 만든 이동 분포
    rng = np.random.default_rng()
    s = rng.poisson(main_data["이동인구(합)"].values)

    main_data["이동인구(합)"] = s

    main_data["출발 행정동 코드"] = list(map(str, main_data["출발 행정동 코드"]))
    main_data["도착 행정동 코드"] = list(map(str, main_data["도착 행정동 코드"]))

    # 이동인구 없는 행 제거
    main_data = main_data.loc[main_data["이동인구(합)"] != 0]

    # main_data -> O-D 수 기준으로 데이터 재구성
    main_data = pd.DataFrame(list(itertools.chain(*[[i.tolist()] * j  for i,j in zip(main_data.values[:,:2], main_data.values[:,2])])), columns = ["origin_code","dest_code"])
    return main_data

In [3]:
main_data = generate_fake_passenger_O_D(5)

In [4]:
#법정동 코드
extradata_ct = connect_container("extradata")
hjd_20180401 = load_json_trans_data("hjd_20180401", extradata_ct, type="gpd")

In [5]:
start = main_data[["origin_code"]]; end = main_data[["dest_code"]]
start.columns = ["adm_cd"]
end.columns = ["adm_cd"]

start = pd.merge(start, hjd_20180401[["adm_cd", "adm_nm"]])
end = pd.merge(end, hjd_20180401[["adm_cd", "adm_nm"]])

main_data = pd.concat([start["adm_nm"], end["adm_nm"]], axis=1)
main_data.columns = ["startpos", "endpos"]

main_data["startpos"] = list(map(lambda data: f"{data.split(' ')[-1]} {data.split(' ')[1]}",main_data["startpos"]))
main_data["endpos"] = list(map(lambda data: f"{data.split(' ')[-1]} {data.split(' ')[1]}",main_data["endpos"]))

In [6]:
main_data = add_ps_location(main_data, mode = "fake")

100%|██████████| 672/672 [04:00<00:00,  2.79it/s]


In [7]:
#승객 데이터 - 콜 시간 분포 알아내기 위함
test = dispatch_data_loader("20220216")

In [8]:
###시간 단위 변환 함수 (0~1440분) 
def time_transform(ps_loc_data):
    ps_loc_data = ps_loc_data[["receipttime_date", "receipttime_time"]]
    
    #고객 콜 접수 시간
    receipttime_date = pd.to_datetime(ps_loc_data['receipttime_date'], format='%Y-%m-%d')
    receipttime_date = list(map(lambda data: data.day, receipttime_date))
    receipttime_time = pd.to_datetime(ps_loc_data['receipttime_time'], format='%H:%M:%S')
    
    #시분초 -> 0~1440분으로 변환
    receipt_time = receipttime_time.dt.minute + receipttime_time.dt.hour*60
    receipt_time = receipt_time.tolist()
    
    ps_loc_data["time"] = [t+1440 if d == 17 else t for t,d in zip(receipt_time, receipttime_date)]
    
    ps_loc_data = ps_loc_data["time"].value_counts().reset_index() 
    ps_loc_data.columns = ["time", "cnt"]
    ps_loc_data["ratio"] = [i/sum(ps_loc_data["cnt"]) for i in ps_loc_data["cnt"]]
    return ps_loc_data

def add_passenger_type(ps_loc_data):
    np.random.seed(1205)
    passenger_type_data = np.random.choice(2 ,size = len(ps_loc_data), p=[0.2, 0.8])
    ps_loc_data["passenger_type"] = passenger_type_data
    return ps_loc_data

In [9]:
a = time_transform(test)

main_data["time"] = np.random.choice(a["time"].tolist() ,size = len(main_data), p= a["ratio"].tolist())

In [10]:
main_data["ps_type"] = np.random.choice(2 ,size = len(main_data), p= [0.2,0.8])

In [13]:
test = main_data[["ps_type", "start_point", "end_point", "time"]]
test.columns = ["ps_type", "start_point", "end_point", "call_time"]
test

,ps_type,start_point,end_point,call_time
0,0,POINT (126.97268158 37.57487244),POINT (126.9788589463704 37.61550086401778),1070
1,1,POINT (126.97338985 37.571609025),POINT (126.95838955 37.60673245),760
2,1,POINT (126.9673256069167 37.57668602238805),POINT (126.969341765616 37.61318027196901),501
3,1,POINT (126.9769507396716 37.56972978815023),POINT (126.9656693811282 37.60757051395395),1031
4,1,POINT (126.9719694 37.57791763333333),POINT (126.9690096142191 37.60897823958356),550
...,...,...,...,...
20672,1,POINT (127.14746358 37.53970078),POINT (127.1257897 37.44387556666666),600
20673,0,POINT (127.1472812333333 37.54161143333333),POINT (127.15200386 37.4514203),1085
20674,1,POINT (127.1429249 37.54099942),POINT (127.1348850666667 37.44488223333334),826
20675,1,POINT (127.140775507064 37.53469481156875),POINT (127.13596945 37.4485153),1780


In [14]:
test = get_route_time_dataframe(test)

In [15]:
test

,ps_type,start_point,end_point,call_time,route,timestamp,distance
0,0,POINT (126.97268158 37.57487244),POINT (126.9788589463704 37.61550086401778),1070,"[[126.97268, 37.57487], [126.97268, 37.57508],...","[0, 0.09, 0.18, 0.27, 0.32, 0.37, 0.42, 0.47, ...",6111.3
1,1,POINT (126.97338985 37.571609025),POINT (126.95838955 37.60673245),760,"[[126.97339, 37.57161], [126.97325, 37.57161],...","[0, 0.11, 0.22, 0.37, 0.52, 0.67, 0.8200000000...",4856.2
2,1,POINT (126.9673256069167 37.57668602238805),POINT (126.969341765616 37.61318027196901),501,"[[126.96733, 37.57669], [126.96742, 37.57674],...","[0, 0.08, 0.16, 0.28, 0.4, 0.52, 0.64, 0.76, 0...",5972.0
3,1,POINT (126.9769507396716 37.56972978815023),POINT (126.9656693811282 37.60757051395395),1031,"[[126.97695, 37.56973], [126.97692, 37.5697], ...","[0, 0.02, 0.04, 0.15, 0.32, 0.4, 0.48000000000...",5955.9
4,1,POINT (126.9719694 37.57791763333333),POINT (126.9690096142191 37.60897823958356),550,"[[126.97197, 37.57792], [126.97196, 37.57794],...","[0, 0.05, 0.1, 0.15000000000000002, 0.2, 0.25,...",4649.0
...,...,...,...,...,...,...,...
20672,1,POINT (127.14746358 37.53970078),POINT (127.1257897 37.44387556666666),600,"[[127.14746, 37.5397], [127.14756, 37.54003], ...","[0, 0.15, 0.3, 0.4, 0.5, 0.6, 0.7, 0.799999999...",12798.6
20673,0,POINT (127.1472812333333 37.54161143333333),POINT (127.15200386 37.4514203),1085,"[[127.14728, 37.54161], [127.14722, 37.54141],...","[0, 0.1, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.5...",12944.9
20674,1,POINT (127.1429249 37.54099942),POINT (127.1348850666667 37.44488223333334),826,"[[127.14293, 37.541], [127.14292, 37.54089], [...","[0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.44, 0...",12581.8
20675,1,POINT (127.140775507064 37.53469481156875),POINT (127.13596945 37.4485153),1780,"[[127.14078, 37.5347], [127.14068, 37.53445], ...","[0, 0.08, 0.15000000000000002, 0.2200000000000...",11806.5
